In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder\
    .master('local[*]')\
    .appName('Classificacao com Spark')\
    .config("spark.driver.bindAddress", "localhost") \
    .config("spark.ui.port", "4050") \
    .config("spark.driver.memory", "4g") \
    .config("spark.driver.host", "localhost") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .getOrCreate()

In [4]:
spark

In [5]:
data = spark.read.csv('Datas', sep=',', header=True, inferSchema= True)

In [6]:
data.toPandas()

,id,Churn,Mais65anos,Conjuge,Dependentes,MesesDeContrato,TelefoneFixo,MaisDeUmaLinhaTelefonica,Internet,SegurancaOnline,BackupOnline,SeguroDispositivo,SuporteTecnico,TVaCabo,StreamingFilmes,TipoContrato,ContaCorreio,MetodoPagamento,MesesCobrados
0,0,Nao,0,Sim,Nao,1,Nao,SemServicoTelefonico,DSL,Nao,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,BoletoEletronico,29.850000
1,1,Nao,0,Nao,Nao,34,Sim,Nao,DSL,Sim,Nao,Sim,Nao,Nao,Nao,UmAno,Nao,Boleto,56.950000
2,2,Sim,0,Nao,Nao,2,Sim,Nao,DSL,Sim,Sim,Nao,Nao,Nao,Nao,Mensalmente,Sim,Boleto,53.850000
3,3,Nao,0,Nao,Nao,45,Nao,SemServicoTelefonico,DSL,Sim,Nao,Sim,Sim,Nao,Nao,UmAno,Nao,DebitoEmConta,42.300000
4,4,Sim,0,Nao,Nao,2,Sim,Nao,FibraOptica,Nao,Nao,Nao,Nao,Nao,Nao,Mensalmente,Sim,BoletoEletronico,70.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10343,10343,Sim,0,Sim,Nao,4,Sim,Sim,FibraOptica,Nao,Nao,Nao,Nao,Sim,Nao,Mensalmente,Sim,BoletoEletronico,86.687604
10344,10344,Sim,1,Nao,Nao,13,Sim,Sim,FibraOptica,Nao,Nao,Nao,Nao,Sim,Nao,Mensalmente,Sim,BoletoEletronico,86.195233
10345,10345,Sim,0,Nao,Nao,15,Sim,Sim,FibraOptica,Nao,Nao,Nao,Nao,Nao,Nao,Mensalmente,Sim,Boleto,75.099071
10346,10346,Sim,0,Nao,Nao,17,Sim,Sim,FibraOptica,Nao,Nao,Nao,Nao,Nao,Sim,Mensalmente,Sim,CartaoCredito,87.824082


In [7]:
data.count()

10348

In [8]:
data.groupBy('Churn').count().withColumnRenamed('count', 'Contagem').show()

+-----+--------+
|Churn|Contagem|
+-----+--------+
|  Sim|    5174|
|  Nao|    5174|
+-----+--------+



In [9]:
data.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Churn: string (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- Conjuge: string (nullable = true)
 |-- Dependentes: string (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- TelefoneFixo: string (nullable = true)
 |-- MaisDeUmaLinhaTelefonica: string (nullable = true)
 |-- Internet: string (nullable = true)
 |-- SegurancaOnline: string (nullable = true)
 |-- BackupOnline: string (nullable = true)
 |-- SeguroDispositivo: string (nullable = true)
 |-- SuporteTecnico: string (nullable = true)
 |-- TVaCabo: string (nullable = true)
 |-- StreamingFilmes: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- ContaCorreio: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)



In [10]:
data.createOrReplaceTempView('dataVIEW')

In [11]:
spark.sql('SELECT INTERNET, COUNT(INTERNET) FROM DATAVIEW GROUP BY 1').show()

+-----------+---------------+
|   INTERNET|count(INTERNET)|
+-----------+---------------+
|FibraOptica|           5401|
|        Nao|           1741|
|        DSL|           3206|
+-----------+---------------+



# Tratando Dados

### Tratando colunas que possuem "Sim" ou "Não" para 1 ou 0

In [12]:
colunasBinarias = [
    'Churn',
    'Conjuge',
    'Dependentes',
    'TelefoneFixo',
    'MaisDeUmaLinhaTelefonica',
    'SegurancaOnline',
    'BackupOnline',
    'SeguroDispositivo',
    'SuporteTecnico',
    'TVaCabo',
    'StreamingFilmes',
    'ContaCorreio'
]

In [13]:
from pyspark.sql import functions as f

##### Criamos uma lista comprehension (um for dentro da lista) para iterar cada dado de uma outra lista, após isso usamos um function do spark para conseguir criar uma condiçao se quando o a coluna tiver o valor 'Sim' vai igual 1 senão vai ser igual 0

In [14]:
todasColunas = [f.when(f.col(c) == 'Sim', 1).otherwise(0).alias(c) for c in colunasBinarias]

In [15]:
todasColunas

[Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS Churn'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS Conjuge'>,
 Column<'CASE WHEN (Dependentes = Sim) THEN 1 ELSE 0 END AS Dependentes'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS TelefoneFixo'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS MaisDeUmaLinhaTelefonica'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS SegurancaOnline'>,
 Column<'CASE WHEN (BackupOnline = Sim) THEN 1 ELSE 0 END AS BackupOnline'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS SeguroDispositivo'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS SuporteTecnico'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS TVaCabo'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS StreamingFilmes'>,
 Column<'CASE WHEN (ContaCorreio = Sim) THEN 1 ELSE 0 END AS ContaCorreio'>]

In [16]:
for coluna in reversed(data.columns):
    if coluna not in colunasBinarias:
        todasColunas.insert(0, coluna)

In [17]:
todasColunas

['id',
 'Mais65anos',
 'MesesDeContrato',
 'Internet',
 'TipoContrato',
 'MetodoPagamento',
 'MesesCobrados',
 Column<'CASE WHEN (Churn = Sim) THEN 1 ELSE 0 END AS Churn'>,
 Column<'CASE WHEN (Conjuge = Sim) THEN 1 ELSE 0 END AS Conjuge'>,
 Column<'CASE WHEN (Dependentes = Sim) THEN 1 ELSE 0 END AS Dependentes'>,
 Column<'CASE WHEN (TelefoneFixo = Sim) THEN 1 ELSE 0 END AS TelefoneFixo'>,
 Column<'CASE WHEN (MaisDeUmaLinhaTelefonica = Sim) THEN 1 ELSE 0 END AS MaisDeUmaLinhaTelefonica'>,
 Column<'CASE WHEN (SegurancaOnline = Sim) THEN 1 ELSE 0 END AS SegurancaOnline'>,
 Column<'CASE WHEN (BackupOnline = Sim) THEN 1 ELSE 0 END AS BackupOnline'>,
 Column<'CASE WHEN (SeguroDispositivo = Sim) THEN 1 ELSE 0 END AS SeguroDispositivo'>,
 Column<'CASE WHEN (SuporteTecnico = Sim) THEN 1 ELSE 0 END AS SuporteTecnico'>,
 Column<'CASE WHEN (TVaCabo = Sim) THEN 1 ELSE 0 END AS TVaCabo'>,
 Column<'CASE WHEN (StreamingFilmes = Sim) THEN 1 ELSE 0 END AS StreamingFilmes'>,
 Column<'CASE WHEN (ContaCorr

In [18]:
data.select(todasColunas).toPandas()

,id,Mais65anos,MesesDeContrato,Internet,TipoContrato,MetodoPagamento,MesesCobrados,Churn,Conjuge,Dependentes,TelefoneFixo,MaisDeUmaLinhaTelefonica,SegurancaOnline,BackupOnline,SeguroDispositivo,SuporteTecnico,TVaCabo,StreamingFilmes,ContaCorreio
0,0,0,1,DSL,Mensalmente,BoletoEletronico,29.850000,0,1,0,0,0,0,1,0,0,0,0,1
1,1,0,34,DSL,UmAno,Boleto,56.950000,0,0,0,1,0,1,0,1,0,0,0,0
2,2,0,2,DSL,Mensalmente,Boleto,53.850000,1,0,0,1,0,1,1,0,0,0,0,1
3,3,0,45,DSL,UmAno,DebitoEmConta,42.300000,0,0,0,0,0,1,0,1,1,0,0,0
4,4,0,2,FibraOptica,Mensalmente,BoletoEletronico,70.700000,1,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10343,10343,0,4,FibraOptica,Mensalmente,BoletoEletronico,86.687604,1,1,0,1,1,0,0,0,0,1,0,1
10344,10344,1,13,FibraOptica,Mensalmente,BoletoEletronico,86.195233,1,0,0,1,1,0,0,0,0,1,0,1
10345,10345,0,15,FibraOptica,Mensalmente,Boleto,75.099071,1,0,0,1,1,0,0,0,0,0,0,1
10346,10346,0,17,FibraOptica,Mensalmente,CartaoCredito,87.824082,1,0,0,1,1,0,0,0,0,0,1,1


In [19]:
dataset = data.select(todasColunas)

In [20]:
dataset.toPandas()

,id,Mais65anos,MesesDeContrato,Internet,TipoContrato,MetodoPagamento,MesesCobrados,Churn,Conjuge,Dependentes,TelefoneFixo,MaisDeUmaLinhaTelefonica,SegurancaOnline,BackupOnline,SeguroDispositivo,SuporteTecnico,TVaCabo,StreamingFilmes,ContaCorreio
0,0,0,1,DSL,Mensalmente,BoletoEletronico,29.850000,0,1,0,0,0,0,1,0,0,0,0,1
1,1,0,34,DSL,UmAno,Boleto,56.950000,0,0,0,1,0,1,0,1,0,0,0,0
2,2,0,2,DSL,Mensalmente,Boleto,53.850000,1,0,0,1,0,1,1,0,0,0,0,1
3,3,0,45,DSL,UmAno,DebitoEmConta,42.300000,0,0,0,0,0,1,0,1,1,0,0,0
4,4,0,2,FibraOptica,Mensalmente,BoletoEletronico,70.700000,1,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10343,10343,0,4,FibraOptica,Mensalmente,BoletoEletronico,86.687604,1,1,0,1,1,0,0,0,0,1,0,1
10344,10344,1,13,FibraOptica,Mensalmente,BoletoEletronico,86.195233,1,0,0,1,1,0,0,0,0,1,0,1
10345,10345,0,15,FibraOptica,Mensalmente,Boleto,75.099071,1,0,0,1,1,0,0,0,0,0,0,1
10346,10346,0,17,FibraOptica,Mensalmente,CartaoCredito,87.824082,1,0,0,1,1,0,0,0,0,0,1,1


In [21]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- Internet: string (nullable = true)
 |-- TipoContrato: string (nullable = true)
 |-- MetodoPagamento: string (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)



# Criação de Dummies

In [22]:
data.select(['Internet', 'TipoContrato', 'MetodoPagamento']).toPandas()

,Internet,TipoContrato,MetodoPagamento
0,DSL,Mensalmente,BoletoEletronico
1,DSL,UmAno,Boleto
2,DSL,Mensalmente,Boleto
3,DSL,UmAno,DebitoEmConta
4,FibraOptica,Mensalmente,BoletoEletronico
...,...,...,...
10343,FibraOptica,Mensalmente,BoletoEletronico
10344,FibraOptica,Mensalmente,BoletoEletronico
10345,FibraOptica,Mensalmente,Boleto
10346,FibraOptica,Mensalmente,CartaoCredito


##### Faremps um pivoteamento agrupando pelo id, onde as linhas se tornaração coluna, usamos o lit para criar linhas onde vai inserir valor 1 se possui correspondência

In [23]:
dataset.groupby('id').pivot('Internet').agg(f.lit(1)).na.fill(0).toPandas()

,id,DSL,FibraOptica,Nao
0,7982,1,0,0
1,9465,0,1,0
2,2122,1,0,0
3,3997,1,0,0
4,6654,0,1,0
...,...,...,...,...
10343,1476,0,0,1
10344,1138,1,0,0
10345,5125,0,1,0
10346,2376,1,0,0


In [24]:
Internet = dataset.groupby('id').pivot('Internet').agg(f.lit(1)).na.fill(0)
TipoContrato = dataset.groupby('id').pivot('TipoContrato').agg(f.lit(1)).na.fill(0)
MetodoPagamento = dataset.groupby('id').pivot('MetodoPagamento').agg(f.lit(1)).na.fill(0)

In [25]:
dataset\
.join(Internet, 'id', how='inner')\
.join(TipoContrato, 'id', how='inner')\
.join(MetodoPagamento, 'id', how='inner')\
.toPandas()

,id,Mais65anos,MesesDeContrato,Internet,TipoContrato,MetodoPagamento,MesesCobrados,Churn,Conjuge,Dependentes,...,DSL,FibraOptica,Nao,DoisAnos,Mensalmente,UmAno,Boleto,BoletoEletronico,CartaoCredito,DebitoEmConta
0,7982,0,1,DSL,Mensalmente,BoletoEletronico,45.305408,1,0,0,...,1,0,0,0,1,0,0,1,0,0
1,9465,0,60,FibraOptica,Mensalmente,BoletoEletronico,103.614223,1,1,0,...,0,1,0,0,1,0,0,1,0,0
2,2122,0,12,DSL,UmAno,Boleto,75.850000,0,0,0,...,1,0,0,0,0,1,1,0,0,0
3,3997,0,69,DSL,DoisAnos,CartaoCredito,61.450000,0,1,0,...,1,0,0,1,0,0,0,0,1,0
4,6654,0,7,FibraOptica,Mensalmente,BoletoEletronico,86.500000,1,1,0,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10343,1476,0,2,Nao,Mensalmente,Boleto,20.250000,0,0,0,...,0,0,1,0,1,0,1,0,0,0
10344,1138,0,11,DSL,Mensalmente,DebitoEmConta,65.150000,0,1,1,...,1,0,0,0,1,0,0,0,0,1
10345,5125,0,11,FibraOptica,Mensalmente,BoletoEletronico,94.000000,1,1,0,...,0,1,0,0,1,0,0,1,0,0
10346,2376,0,11,DSL,UmAno,BoletoEletronico,61.250000,0,0,0,...,1,0,0,0,0,1,0,1,0,0


### Renomenado as Colunas e romovendo outras

In [26]:
dataset\
.join(Internet, 'id', how='inner')\
.join(TipoContrato, 'id', how='inner')\
.join(MetodoPagamento, 'id', how='inner')\
.select(
        '*',
        f.col('DSL').alias('Internet_DSL'), 
        f.col('FibraOptica').alias('Internet_FibraOptica'), 
        f.col('Nao').alias('Internet_Nao'), 
        f.col('Mensalmente').alias('TipoContrato_Mensalmente'), 
        f.col('UmAno').alias('TipoContrato_UmAno'), 
        f.col('DoisAnos').alias('TipoContrato_DoisAnos'), 
        f.col('DebitoEmConta').alias('MetodoPagamento_DebitoEmConta'), 
        f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'), 
        f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'), 
        f.col('Boleto').alias('MetodoPagamento_Boleto')        
    )\
.drop(
    'Internet', 'TipoContrato', 'MetodoPagamento', 'DSL', 
    'FibraOptica', 'Nao', 'Mensalmente', 'UmAno', 'DoisAnos', 
    'DebitoEmConta', 'CartaoCredito', 'BoletoEletronico', 'Boleto'
)\
.toPandas()

,id,Mais65anos,MesesDeContrato,MesesCobrados,Churn,Conjuge,Dependentes,TelefoneFixo,MaisDeUmaLinhaTelefonica,SegurancaOnline,...,Internet_DSL,Internet_FibraOptica,Internet_Nao,TipoContrato_Mensalmente,TipoContrato_UmAno,TipoContrato_DoisAnos,MetodoPagamento_DebitoEmConta,MetodoPagamento_CartaoCredito,MetodoPagamento_BoletoEletronico,MetodoPagamento_Boleto
0,7982,0,1,45.305408,1,0,0,0,0,0,...,1,0,0,1,0,0,0,0,1,0
1,9465,0,60,103.614223,1,1,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
2,2122,0,12,75.850000,0,0,0,1,1,0,...,1,0,0,0,1,0,0,0,0,1
3,3997,0,69,61.450000,0,1,0,1,0,0,...,1,0,0,0,0,1,0,1,0,0
4,6654,0,7,86.500000,1,1,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10343,1476,0,2,20.250000,0,0,0,1,0,0,...,0,0,1,1,0,0,0,0,0,1
10344,1138,0,11,65.150000,0,1,1,1,1,1,...,1,0,0,1,0,0,1,0,0,0
10345,5125,0,11,94.000000,1,1,0,1,1,0,...,0,1,0,1,0,0,0,0,1,0
10346,2376,0,11,61.250000,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,1,0


In [27]:
dataset = dataset\
.join(Internet, 'id', how='inner')\
.join(TipoContrato, 'id', how='inner')\
.join(MetodoPagamento, 'id', how='inner')\
.select(
        '*',
        f.col('DSL').alias('Internet_DSL'), 
        f.col('FibraOptica').alias('Internet_FibraOptica'), 
        f.col('Nao').alias('Internet_Nao'), 
        f.col('Mensalmente').alias('TipoContrato_Mensalmente'), 
        f.col('UmAno').alias('TipoContrato_UmAno'), 
        f.col('DoisAnos').alias('TipoContrato_DoisAnos'), 
        f.col('DebitoEmConta').alias('MetodoPagamento_DebitoEmConta'), 
        f.col('CartaoCredito').alias('MetodoPagamento_CartaoCredito'), 
        f.col('BoletoEletronico').alias('MetodoPagamento_BoletoEletronico'), 
        f.col('Boleto').alias('MetodoPagamento_Boleto')        
    )\
.drop(
    'Internet', 'TipoContrato', 'MetodoPagamento', 'DSL', 
    'FibraOptica', 'Nao', 'Mensalmente', 'UmAno', 'DoisAnos', 
    'DebitoEmConta', 'CartaoCredito', 'BoletoEletronico', 'Boleto'
)

In [28]:
dataset.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Mais65anos: integer (nullable = true)
 |-- MesesDeContrato: integer (nullable = true)
 |-- MesesCobrados: double (nullable = true)
 |-- Churn: integer (nullable = false)
 |-- Conjuge: integer (nullable = false)
 |-- Dependentes: integer (nullable = false)
 |-- TelefoneFixo: integer (nullable = false)
 |-- MaisDeUmaLinhaTelefonica: integer (nullable = false)
 |-- SegurancaOnline: integer (nullable = false)
 |-- BackupOnline: integer (nullable = false)
 |-- SeguroDispositivo: integer (nullable = false)
 |-- SuporteTecnico: integer (nullable = false)
 |-- TVaCabo: integer (nullable = false)
 |-- StreamingFilmes: integer (nullable = false)
 |-- ContaCorreio: integer (nullable = false)
 |-- Internet_DSL: integer (nullable = true)
 |-- Internet_FibraOptica: integer (nullable = true)
 |-- Internet_Nao: integer (nullable = true)
 |-- TipoContrato_Mensalmente: integer (nullable = true)
 |-- TipoContrato_UmAno: integer (nullable = true)
 |-- TipoContr

In [34]:
dataset.write.parquet('Dataset Modelo/',
                     mode= 'overwrite')